In [14]:
from utils import Utils, AOC
from session import SESSION
from collections import defaultdict
from matplotlib import pyplot as plt
import numpy as np
import sortedcontainers
import time

In [2]:
aoc = AOC(session=SESSION)

In [3]:
aoc.verify_session()

True

In [4]:
aoc.get_today_file().analyse().head()

Local file found.
100% of data are digits. Analyse as numbers.
0 empty line(s) found. Analyse as monline data.
===== HEAD (5/100) =====
8278572114793756191833561127452853426929615899116958969481427593944191389121411192127899512211639555
2495171585122426519281919685613271991924121543192937558138395964783255375528718245475125436917188379
7961821999883965642974131223415952226122149135757469151378271261942142914126999286296912626324385126
6441179418166119485338911954311735412134219517651998824469721124695113922376591228278111534398748579
8424516666869191815919915114592841194492325941357613456911196292486835511835219129622871119578589971


In [5]:
def new_line(line):
    nl = []
    for i in range(5):
        for el in line:
            nl.append(((int(el) + i - 1) % 9) + 1)
    return nl

In [6]:
def extend_row(row):
    final_raw = ''
    for i in range(5):
        for line in raw.split('\n'):
            for el in line:
                if el.isdigit():
                    final_raw += str(((int(el) + i - 1) % 9) + 1)
            final_raw += '\n'

    return final_raw[:-1]

In [7]:
raw = """1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581"""

In [8]:
raw = aoc.raw

In [9]:
grid = np.array([new_line(line) for line in extend_row(raw).split('\n')])

In [10]:
grid.mean()

4.990596

In [11]:
def reconstruct_path(came_from, current):
    path = [current]
    while current in came_from:
        current = came_from[current]
        path.insert(0, current)
    return path

In [15]:
def a_star(start, goal):
    open_set = {start}
    came_from = {}

    # known
    g_score = defaultdict(lambda: float('inf'))
    g_score[start] = 0

    # guess
    f_score = defaultdict(lambda: float('inf'))
    f_score[start] = Utils.coordinates_distance(start, goal)

    while open_set:
        _, current = sortedcontainers.SortedDict({f_score[el]: el for el in open_set}).popitem(0)

        if current == goal:
            return reconstruct_path(came_from, current)

        open_set.remove(current)

        for neighbour in Utils.get_neighbours(current, dimensions=((0, len(grid)-1), (0, len(grid)-1))):
            tentative = g_score[current] + grid[neighbour]

            if tentative < g_score[neighbour]:
                came_from[neighbour] = current
                g_score[neighbour] = tentative
                f_score[neighbour] = tentative + Utils.coordinates_distance(current, goal) * 2

                open_set.add(neighbour)

In [17]:
# != 3209
# == 2846

In [18]:
st = time.time()
path = a_star((0, 0), (len(grid)-1, len(grid)-1))
print(f'{time.time() - st:.2f}s')
print(f'{sum([grid[el] for el in path[1:]]):.2f}')

67.66s
2846.00


## Post submission optimisation
- new optimised `a_star` algorithm in `Utils.Graph`.

In [29]:
st = time.time()
multiplicator = int(grid.mean()/2)
opt_path = Utils.a_star(
    start=(0, 0),
    goal=(len(grid)-1, len(grid)-1),
    estimate=lambda x,y: (abs(x[0]-y[0]) + abs(x[1]-y[1])) * multiplicator,
    weight=lambda f,t: grid[t],
    get_neighbours=lambda co: Utils.get_neighbours(co, dimensions=((0, len(grid)-1), (0, len(grid)-1)))
)
print(f'{time.time() - st:.2f}s')
print(f'{sum([grid[el] for el in opt_path[1:]]):.2f}')

7.36s
2846.00
